In [35]:
from astropy.io import fits
from astropy.table import Table
import numpy as np
import matplotlib.pylab as plt 
import matplotlib.lines as mlines
from matplotlib.legend import Legend
from pythonds.basic.stack import Stack
from math import *
from sklearn.neighbors import KDTree
from lrg_plot_functions import *
from lrg_sum_functions import *
from cosmo_Calc import *
from divideByTwo import *
# from read_data import *
from nearNeighbors import *
from scipy import stats
from astropy import stats
import healpy as hp
from localBKG_and_interlopersHEALPix import *
# import healpy as hpy
from makeCutoutTableFunc import *
from plotHistogramProjections3D import *
from read_bricks import *
from mpl_toolkits.axes_grid1 import make_axes_locatable
import random

In [36]:
def read_data_new(SpecObj_data_south, SDSS_data, DECaLS_data):
    # Read in data from SDSS file

    # Redshift of galaxies according to sdss
    z = []
    z = SDSS_data.field('Z')

    # Unique ID for sources in SDSS
    specobjid = []
    specobjid = SDSS_data.field('SPECOBJID')

    # Class of object
    gal_class = []
    gal_class = SDSS_data.field('CLASS')

    # What survey the data is from
    survey = []
    survey = SDSS_data.field('SURVEY')

    # SPECPRIMARY; set to 1 for primary observation of object, 0 otherwise
    spec = []
    spec = SDSS_data.field('SPECPRIMARY')

    # Bitmask of spectroscopic warning values; need set to 0
    zwarn_noqso = []
    zwarn_noqso = SDSS_data.field('ZWARNING_NOQSO')

    # Spectroscopic classification for certain redshift?
    class_noqso = []
    class_noqso = SDSS_data.field('CLASS_NOQSO')

    # Array for LRG targets
    targets = []
    targets = SDSS_data.field('BOSS_TARGET1')

    # Array for tile number (cut for LOWZ LRGs is tile >= 10324)
    tile = []
    tile = SDSS_data.field('TILE')

    # Array for chunk number (cut for CMASS is chunk != 'boss1' or 'boss2'
    chunk = []
    chunk = SDSS_data.field('CHUNK')

    plate = []
    plate = SDSS_data.field('PLATEID')

    fiberid = []
    fiberid = SDSS_data.field('FIBERID')


    print(len(z))
    print('done reading in SDSS')
    
    
    # Read in data from SDSS row matched DECaLS file

    # release 
    release_MATCHED = []
    release_MATCHED = SpecObj_data.field('release')

    # Object ID from survey file; value -1 for non-matches
    objid_MATCHED = []
    objid_MATCHED = SpecObj_data.field('OBJID')

    # Add brickid
    brickid_MATCHED = []
    brickid_MATCHED = SpecObj_data.field('BRICKID')

    # Add brickname
    brickname_MATCHED = []
    brickname_MATCHED = SpecObj_data.field('BRICKNAME')

    # Only galaxies included
    gal_type_MATCHED = []
    gal_type_MATCHED = SpecObj_data.field('TYPE')

    # RA
    ra_MATCHED = []
    ra_MATCHED = SpecObj_data.field('RA')

    # Dec
    dec_MATCHED = []
    dec_MATCHED = SpecObj_data.field('DEC')

    # flux_g
    gflux_MATCHED = []
    gflux_MATCHED = SpecObj_data.field('FLUX_G')

    # flux_r
    rflux_MATCHED = []
    rflux_MATCHED = SpecObj_data.field('FLUX_R')

    # flux_z
    zflux_MATCHED = []
    zflux_MATCHED = SpecObj_data.field('FLUX_Z')

    # flux_W1
    w1flux_MATCHED = []
    w1flux_MATCHED = SpecObj_data.field('flux_w1')

    # flux_W2
    w2flux_MATCHED = []
    w2flux_MATCHED = SpecObj_data.field('flux_w2')

    # flux_W3
    w3flux_MATCHED = []
    w3flux_MATCHED = SpecObj_data.field('flux_w3')

    # flux_W4
    w4flux_MATCHED = []
    w4flux_MATCHED = SpecObj_data.field('flux_w4')

    # nobs == number of images that contribute to the central pixel
    # nobs_g
    gobs_MATCHED = []
    gobs_MATCHED = SpecObj_data.field('NOBS_G')

    # nobs_r
    robs_MATCHED = []
    robs_MATCHED = SpecObj_data.field('NOBS_R')

    # nobs_z
    zobs_MATCHED = []
    zobs_MATCHED = SpecObj_data.field('NOBS_Z')

    #flux errors
    gflux_ivar_MATCHED = []
    gflux_ivar_MATCHED = SpecObj_data.field('flux_ivar_g')

    rflux_ivar_MATCHED = []
    rflux_ivar_MATCHED = SpecObj_data.field('flux_ivar_r')

    zflux_ivar_MATCHED = []
    zflux_ivar_MATCHED = SpecObj_data.field('flux_ivar_z')

    w1flux_ivar_MATCHED = []
    w1flux_ivar_MATCHED = SpecObj_data.field('flux_ivar_w1')

    w2flux_ivar_MATCHED = []
    w2flux_ivar_MATCHED = SpecObj_data.field('flux_ivar_w2')

    w3flux_ivar_MATCHED = []
    w3flux_ivar_MATCHED = SpecObj_data.field('flux_ivar_w3')

    w4flux_ivar_MATCHED = []
    w4flux_ivar_MATCHED = SpecObj_data.field('flux_ivar_w4')

    mw_transmission_g_MATCHED = []
    mw_transmission_g_MATCHED = SpecObj_data.field('mw_transmission_g')

    mw_transmission_r_MATCHED = []
    mw_transmission_r_MATCHED = SpecObj_data.field('mw_transmission_r')

    mw_transmission_z_MATCHED = []
    mw_transmission_z_MATCHED = SpecObj_data.field('mw_transmission_z')

    mw_transmission_w1_MATCHED = []
    mw_transmission_w1_MATCHED = SpecObj_data.field('mw_transmission_w1')

    mw_transmission_w2_MATCHED = []
    mw_transmission_w2_MATCHED = SpecObj_data.field('mw_transmission_w2')

    mw_transmission_w3_MATCHED = []
    mw_transmission_w3_MATCHED = SpecObj_data.field('mw_transmission_w3')

    mw_transmission_w4_MATCHED = []
    mw_transmission_w4_MATCHED = SpecObj_data.field('mw_transmission_w4')

    # maskbits
    maskbits_MATCHED = []
    maskbits_MATCHED = SpecObj_data.field('maskbits')
    
    galdepth_g_MATCHED = []
    galdepth_g_MATCHED = SpecObj_data.field('galdepth_g')
    
    galdepth_r_MATCHED = []
    galdepth_r_MATCHED = SpecObj_data.field('galdepth_r')
    
    galdepth_z_MATCHED = []
    galdepth_z_MATCHED = SpecObj_data.field('galdepth_z')
    
    psfdepth_g_MATCHED = []
    psfdepth_g_MATCHED = SpecObj_data.field('psfdepth_g')
    
    psfdepth_r_MATCHED = []
    psfdepth_r_MATCHED = SpecObj_data.field('psfdepth_r')
    
    psfdepth_z_MATCHED = []
    psfdepth_z_MATCHED = SpecObj_data.field('psfdepth_z')


#     print(len(objid_MATCHED[np.where(objid_MATCHED != -1)]))
    print(len(objid_MATCHED))
    print('done reading in DECaLS-SDSS matched file')
    
    
    # Read in data from DECaLS bricks

    # release 
    release_ALL = []
    release_ALL = DECaLS_data.field('release')

    # Object ID from survey file
    objid_ALL = []
    objid_ALL = DECaLS_data.field('OBJID')
    # print(len(objid_ALL))

    # Add brickid
    brickid_ALL = []
    brickid_ALL = DECaLS_data.field('BRICKID')

    # Add brickname
    brickname_ALL = []
    brickname_ALL = DECaLS_data.field('BRICKNAME')

    # Only galaxies included
    gal_type_ALL = []
    gal_type_ALL = DECaLS_data.field('TYPE')

    # RA
    ra_ALL = []
    ra_ALL = DECaLS_data.field('RA')

    # Dec
    dec_ALL = []
    dec_ALL = DECaLS_data.field('DEC')

    # flux_g
    gflux_ALL = []
    gflux_ALL = DECaLS_data.field('FLUX_G')

    # flux_r
    rflux_ALL = []
    rflux_ALL = DECaLS_data.field('FLUX_R')

    # flux_z
    zflux_ALL = []
    zflux_ALL = DECaLS_data.field('FLUX_Z')

    # flux_W1
    w1flux_ALL = []
    w1flux_ALL = DECaLS_data.field('flux_w1')

    # flux_W2
    w2flux_ALL = []
    w2flux_ALL = DECaLS_data.field('flux_w2')

    # flux_W3
    w3flux_ALL = []
    w3flux_ALL = DECaLS_data.field('flux_w3')

    # flux_W4
    w4flux_ALL = []
    w4flux_ALL = DECaLS_data.field('flux_w4')

    # nobs == number of images that contribute to the central pixel
    # nobs_g
    gobs_ALL = []
    gobs_ALL = DECaLS_data.field('NOBS_G')

    # nobs_r
    robs_ALL = []
    robs_ALL = DECaLS_data.field('NOBS_R')

    # nobs_z
    zobs_ALL = []
    zobs_ALL = DECaLS_data.field('NOBS_Z')

    # flux errors
    gflux_ivar_ALL = []
    gflux_ivar_ALL = DECaLS_data.field('flux_ivar_g')

    rflux_ivar_ALL = []
    rflux_ivar_ALL = DECaLS_data.field('flux_ivar_r')

    zflux_ivar_ALL = []
    zflux_ivar_ALL = DECaLS_data.field('flux_ivar_z')

    w1flux_ivar_ALL = []
    w1flux_ivar_ALL = DECaLS_data.field('flux_ivar_w1')

    w2flux_ivar_ALL = []
    w2flux_ivar_ALL = DECaLS_data.field('flux_ivar_w2')

    w3flux_ivar_ALL = []
    w3flux_ivar_ALL = DECaLS_data.field('flux_ivar_w3')

    w4flux_ivar_ALL = []
    w4flux_ivar_ALL = DECaLS_data.field('flux_ivar_w4')

    mw_transmission_g_ALL = []
    mw_transmission_g_ALL = DECaLS_data.field('mw_transmission_g')

    mw_transmission_r_ALL = []
    mw_transmission_r_ALL = DECaLS_data.field('mw_transmission_r')

    mw_transmission_z_ALL = []
    mw_transmission_z_ALL = DECaLS_data.field('mw_transmission_z')

    mw_transmission_w1_ALL = []
    mw_transmission_w1_ALL = DECaLS_data.field('mw_transmission_w1')

    mw_transmission_w2_ALL = []
    mw_transmission_w2_ALL = DECaLS_data.field('mw_transmission_w2')

    mw_transmission_w3_ALL = []
    mw_transmission_w3_ALL = DECaLS_data.field('mw_transmission_w3')

    mw_transmission_w4_ALL = []
    mw_transmission_w4_ALL = DECaLS_data.field('mw_transmission_w4')

    # maskbits
    maskbits_ALL = []
    maskbits_ALL = DECaLS_data.field('maskbits')
    
    galdepth_g_ALL = []
    galdepth_g_ALL = DECaLS_data.field('galdepth_g')
    
    galdepth_r_ALL = []
    galdepth_r_ALL = DECaLS_data.field('galdepth_r')
    
    galdepth_z_ALL = []
    galdepth_z_ALL = DECaLS_data.field('galdepth_z')
    
    psfdepth_g_ALL = []
    psfdepth_g_ALL = DECaLS_data.field('psfdepth_g')
    
    psfdepth_r_ALL = []
    psfdepth_r_ALL = DECaLS_data.field('psfdepth_r')
    
    psfdepth_z_ALL = []
    psfdepth_z_ALL = DECaLS_data.field('psfdepth_z')


    print(len(objid_ALL))
    print('done reading in DECaLS bricks')
    
    
    # extract LRGs from MATCHED file
    
    # Create a unique identifier by combinding BRICKID and OBJID for row-matched file

    id_MATCHED = []

    for i in range(len(objid_MATCHED)):
        if (objid_MATCHED[i] == -1):
            id_MATCHED.append(-1)
        else:
            temp1 = str(release_MATCHED[i]) + str(brickid_MATCHED[i]) + str(objid_MATCHED[i])
            id_MATCHED.append(temp1)

    id_MATCHED = [int(i) for i in id_MATCHED]
    id_MATCHED = np.array(id_MATCHED)


    print(len(id_MATCHED))
    print('done creating unique IDs for matched file')
    
    
    # LRG-only criteria

    LRG_only = ((((targets & 2**0)!=0) | ((targets & 2**1)!=0)) & (gobs_MATCHED >= 2.) & (robs_MATCHED >= 2.) & (zobs_MATCHED >= 2.) & (gflux_MATCHED > 0.) & (rflux_MATCHED > 0.) & (zflux_MATCHED > 0.) & (objid_MATCHED != -1) & ((gal_type_MATCHED == 'SER') | (gal_type_MATCHED == "DEV") | (gal_type_MATCHED == "EXP") | (gal_type_MATCHED == "REX")) & (tile >= 10324) & (gal_class == 'GALAXY') & (spec == 1) & (zwarn_noqso == 0) & (class_noqso == 'GALAXY') & ((survey == 'sdss') | (survey == 'boss')))

    z_LRG = z[LRG_only]
    ra_LRG = ra_MATCHED[LRG_only]
    dec_LRG = dec_MATCHED[LRG_only]
    gflux_LRG = gflux_MATCHED[LRG_only]
    rflux_LRG = rflux_MATCHED[LRG_only]
    zflux_LRG = zflux_MATCHED[LRG_only]
    w1flux_LRG = w1flux_MATCHED[LRG_only]
    w2flux_LRG = w2flux_MATCHED[LRG_only]
    w3flux_LRG = w3flux_MATCHED[LRG_only]
    w4flux_LRG = w4flux_MATCHED[LRG_only]
    gflux_ivar_LRG = gflux_ivar_MATCHED[LRG_only]
    rflux_ivar_LRG = rflux_ivar_MATCHED[LRG_only]
    zflux_ivar_LRG = zflux_ivar_MATCHED[LRG_only]
    w1flux_ivar_LRG = w1flux_ivar_MATCHED[LRG_only]
    w2flux_ivar_LRG = w2flux_ivar_MATCHED[LRG_only]
    w3flux_ivar_LRG = w3flux_ivar_MATCHED[LRG_only]
    w4flux_ivar_LRG = w4flux_ivar_MATCHED[LRG_only]
    id_LRG = id_MATCHED[LRG_only]
    specobjid_LRG = specobjid[LRG_only]
    mw_transmission_g_LRG = mw_transmission_g_MATCHED[LRG_only]
    mw_transmission_r_LRG = mw_transmission_r_MATCHED[LRG_only]
    mw_transmission_z_LRG = mw_transmission_z_MATCHED[LRG_only]
    mw_transmission_w1_LRG = mw_transmission_w1_MATCHED[LRG_only]
    mw_transmission_w2_LRG = mw_transmission_w2_MATCHED[LRG_only]
    mw_transmission_w3_LRG = mw_transmission_w3_MATCHED[LRG_only]
    mw_transmission_w4_LRG = mw_transmission_w4_MATCHED[LRG_only]
    maskbits_LRG = maskbits_MATCHED[LRG_only]
    galdepth_g_LRG = galdepth_g_MATCHED[LRG_only]
    galdepth_r_LRG = galdepth_r_MATCHED[LRG_only]
    galdepth_z_LRG = galdepth_z_MATCHED[LRG_only]
    psfdepth_g_LRG = psfdepth_g_MATCHED[LRG_only]
    psfdepth_r_LRG = psfdepth_r_MATCHED[LRG_only]
    psfdepth_z_LRG = psfdepth_z_MATCHED[LRG_only]
    
    
    # remove LRGs from ALL file
    
    # Create a unique identifier by combinding BRICKID and OBJID for DECaLS brick

    id_ALL = []

    for i in range(len(objid_ALL)):
        temp2 = str(release_ALL[i]) + str(brickid_ALL[i]) + str(objid_ALL[i])
        id_ALL.append(temp2)

    id_ALL = [int(i) for i in id_ALL]
    id_ALL = np.array(id_ALL)

    print('length of id_ALL: ', len(id_ALL))
    print('done creating unique IDs for brick file')
    
    
    # find all instances of LRGs in ALL file
    
    id_cut = id_ALL
    
    array, ind_ALL, ind_LRG = np.intersect1d(id_ALL, id_LRG, return_indices=True)
    print(len(ind_ALL)) # ind_ALL gives the indices of id_ALL where id_ALL have common values with id_LRG
    id_cut[ind_ALL] = 0

    print(len(id_cut))
    print(len(id_cut[np.where(id_ALL == 0)]))
    
    
    survey_cut = ((id_cut != 0) & (gobs_ALL >= 2.) & (robs_ALL >= 2.) & (zobs_ALL >= 2.) & (gflux_ALL > 0.) & (rflux_ALL > 0.) & (zflux_ALL > 0.) & ((gal_type_ALL == 'SER') | (gal_type_ALL == "DEV") | (gal_type_ALL == "EXP") | (gal_type_ALL == "REX")))


#     ra_BKG = ra_ALL[survey_cut]
#     dec_BKG = dec_ALL[survey_cut]
#     gflux_BKG = gflux_ALL[survey_cut]
#     rflux_BKG = rflux_ALL[survey_cut]
#     zflux_BKG = zflux_ALL[survey_cut]
#     w1flux_BKG = w1flux_ALL[survey_cut]
#     w2flux_BKG = w2flux_ALL[survey_cut]
#     w3flux_BKG = w3flux_ALL[survey_cut]
#     w4flux_BKG = w4flux_ALL[survey_cut]
#     gflux_ivar_BKG = gflux_ivar_ALL[survey_cut]
#     rflux_ivar_BKG = rflux_ivar_ALL[survey_cut]
#     zflux_ivar_BKG = zflux_ivar_ALL[survey_cut]
#     w1flux_ivar_BKG = w1flux_ivar_ALL[survey_cut]
#     w2flux_ivar_BKG = w2flux_ivar_ALL[survey_cut]
#     w3flux_ivar_BKG = w3flux_ivar_ALL[survey_cut]
#     w4flux_ivar_BKG = w4flux_ivar_ALL[survey_cut]
#     mw_transmission_g_BKG = mw_transmission_g_ALL[survey_cut]
#     mw_transmission_r_BKG = mw_transmission_r_ALL[survey_cut]
#     mw_transmission_z_BKG = mw_transmission_z_ALL[survey_cut]
#     mw_transmission_w1_BKG = mw_transmission_w1_ALL[survey_cut]
#     mw_transmission_w2_BKG = mw_transmission_w2_ALL[survey_cut]
#     mw_transmission_w3_BKG = mw_transmission_w3_ALL[survey_cut]
#     mw_transmission_w4_BKG = mw_transmission_w4_ALL[survey_cut]
#     maskbits_BKG = maskbits_ALL[survey_cut]
    galdepth_g_BKG = galdepth_g_ALL[survey_cut]
    galdepth_r_BKG = galdepth_r_ALL[survey_cut]
    galdepth_z_BKG = galdepth_z_ALL[survey_cut]
#     psfdepth_g_BKG = psfdepth_g_ALL[survey_cut]
#     psfdepth_r_BKG = psfdepth_r_ALL[survey_cut]
#     psfdepth_z_BKG = psfdepth_z_ALL[survey_cut]
    
#     return psfdepth_g_LRG, psfdepth_r_LRG, psfdepth_z_LRG, psfdepth_g_BKG, psfdepth_r_BKG, psfdepth_z_BKG, galdepth_g_LRG, galdepth_r_LRG, galdepth_z_LRG, galdepth_g_BKG, galdepth_r_BKG, galdepth_z_BKG, maskbits_LRG, maskbits_BKG, id_LRG, ra_LRG, dec_LRG, ra_BKG, dec_BKG, z_LRG, gflux_LRG, rflux_LRG, zflux_LRG, w1flux_LRG, w2flux_LRG, w3flux_LRG, w4flux_LRG, gflux_BKG, rflux_BKG, zflux_BKG, w1flux_BKG, w2flux_BKG, w3flux_BKG, w4flux_BKG, gflux_ivar_LRG, rflux_ivar_LRG, zflux_ivar_LRG, w1flux_ivar_LRG, w2flux_ivar_LRG, w3flux_ivar_LRG, w4flux_ivar_LRG, gflux_ivar_BKG, rflux_ivar_BKG, zflux_ivar_BKG, w1flux_ivar_BKG, w2flux_ivar_BKG, w3flux_ivar_BKG, w4flux_ivar_BKG, mw_transmission_g_LRG, mw_transmission_r_LRG, mw_transmission_z_LRG, mw_transmission_g_BKG, mw_transmission_r_BKG, mw_transmission_z_BKG, mw_transmission_w1_LRG, mw_transmission_w2_LRG, mw_transmission_w3_LRG, mw_transmission_w4_LRG, mw_transmission_w1_BKG, mw_transmission_w2_BKG, mw_transmission_w3_BKG, mw_transmission_w4_BKG
    
#     return psfdepth_g_BKG, psfdepth_r_BKG, psfdepth_z_BKG, galdepth_g_BKG, galdepth_r_BKG, galdepth_z_BKG, maskbits_BKG, ra_BKG, dec_BKG, gflux_BKG, rflux_BKG, zflux_BKG, w1flux_BKG, w2flux_BKG, w3flux_BKG, w4flux_BKG, gflux_ivar_BKG, rflux_ivar_BKG, zflux_ivar_BKG, w1flux_ivar_BKG, w2flux_ivar_BKG, w3flux_ivar_BKG, w4flux_ivar_BKG, mw_transmission_g_BKG, mw_transmission_r_BKG, mw_transmission_z_BKG, mw_transmission_w1_BKG, mw_transmission_w2_BKG, mw_transmission_w3_BKG, mw_transmission_w4_BKG
    return galdepth_g_BKG, galdepth_r_BKG, galdepth_z_BKG



In [37]:
# read in data from fits files

hdu_north = fits.open('/Users/mtownsend/anaconda/Data/survey-dr9-north-specObj-dr16.fits') # this matches SDSS LRGs to MzLS/BASS;                                                               # ONLY GIVES SOURCES THAT ARE IN SDSS and LEGACY SURVEYS
hdu_south = fits.open('/Users/mtownsend/anaconda/Data/survey-dr9-south-specObj-dr16.fits') # this matches SDSS LRGs to DECaLS;
                                                                 # ONLY GIVES SOURCES THAT ARE IN SDSS and LEGACY SURVEYS
hdu_sdss = fits.open('/Users/mtownsend/anaconda/Data/specObj-dr16.fits') # this is SDSS redshifts etc for LRGs


In [39]:
hdu_sweep1 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-130m005-140p000.fits') # this is one sweep file of the DECaLS data
hdu_sweep2 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-130m010-140m005.fits') # this is one sweep file of the DECaLS data
hdu_sweep3 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-130m015-140m010.fits') 
hdu_sweep4 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-130p010-140p015.fits')
hdu_sweep5 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-130p015-140p020.fits')

hdu_sweep6 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-130p020-140p025.fits')
hdu_sweep7 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-130p025-140p030.fits')
hdu_sweep8 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-130p035-140p040.fits')
hdu_sweep9 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-140m005-150p000.fits')
hdu_sweep10 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-140m010-150m005.fits')

hdu_sweep11 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-140m015-150m010.fits') 
hdu_sweep12 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-140p000-150p005.fits')
hdu_sweep13 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-140p005-150p010.fits') 
hdu_sweep14 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-140p010-150p015.fits')
hdu_sweep15 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-140p015-150p020.fits')

hdu_sweep16 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-140p020-150p025.fits') 
hdu_sweep17 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-140p025-150p030.fits') 
hdu_sweep18 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/130-150/sweep-140p030-150p035.fits')
# hdu_sweep19 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/0-80/') 
# hdu_sweep20 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/0-80/')

# hdu_sweep21 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/0-80/')
# hdu_sweep22 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/0-80/') 
# hdu_sweep23 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/0-80/') 
# hdu_sweep24 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/0-80/')
# hdu_sweep25 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/0-80/') 

# hdu_sweep26 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/0-80/') 
# hdu_sweep27 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/0-80/')
# hdu_sweep28 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/0-80/')

# hdu_sweep29 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/100-200/')


In [40]:
SpecObj_data_north = hdu_north[1].data
print('done')
SpecObj_data = hdu_south[1].data
print('done')
SDSS_data = hdu_sdss[1].data
print('done')
DECaLS_data1 = hdu_sweep1[1].data
print('done')
DECaLS_data2 = hdu_sweep2[1].data
print('done')
DECaLS_data3 = hdu_sweep3[1].data
print('done')
DECaLS_data4 = hdu_sweep4[1].data
print('done')
DECaLS_data5 = hdu_sweep5[1].data
print('done')
DECaLS_data6 = hdu_sweep6[1].data
print('done')
DECaLS_data7 = hdu_sweep7[1].data
print('done')
DECaLS_data8 = hdu_sweep8[1].data
print('done')
DECaLS_data9 = hdu_sweep9[1].data
print('done')
DECaLS_data10 = hdu_sweep10[1].data
print('done')
DECaLS_data11 = hdu_sweep11[1].data
print('done')
DECaLS_data12 = hdu_sweep12[1].data
print('done')
DECaLS_data13 = hdu_sweep13[1].data
print('done')
DECaLS_data14 = hdu_sweep14[1].data
print('done')
DECaLS_data15 = hdu_sweep15[1].data
print('done')
DECaLS_data16 = hdu_sweep16[1].data
print('done')
DECaLS_data17 = hdu_sweep17[1].data
print('done')
DECaLS_data18 = hdu_sweep18[1].data
print('done')
# DECaLS_data19 = hdu_sweep19[1].data
# print('done')
# DECaLS_data20 = hdu_sweep20[1].data
# print('done')
# DECaLS_data21 = hdu_sweep21[1].data
# print('done')
# DECaLS_data22 = hdu_sweep22[1].data
# print('done')
# DECaLS_data23 = hdu_sweep23[1].data
# print('done')
# DECaLS_data24 = hdu_sweep24[1].data
# print('done')
# DECaLS_data25 = hdu_sweep25[1].data
# print('done')
# DECaLS_data26 = hdu_sweep26[1].data
# print('done')
# DECaLS_data27 = hdu_sweep27[1].data
# print('done')
# DECaLS_data28 = hdu_sweep28[1].data
# print('done')
# DECaLS_data29 = hdu_sweep29[1].data
# print('done')
# DECaLS_data30 = hdu_sweep30[1].data
# print('done')
# DECaLS_data31 = hdu_sweep31[1].data
# print('done')
# DECaLS_data32 = hdu_sweep32[1].data
# print('done')
# DECaLS_data33 = hdu_sweep33[1].data
# print('done')
# DECaLS_data34 = hdu_sweep34[1].data
# print('done')
# DECaLS_data35 = hdu_sweep35[1].data
# print('done')
# DECaLS_data36 = hdu_sweep36[1].data
# print('done')
# DECaLS_data37 = hdu_sweep37[1].data
# print('done')
# DECaLS_data38 = hdu_sweep38[1].data
# print('done')
# DECaLS_data39 = hdu_sweep39[1].data
# print('done')
# DECaLS_data40 = hdu_sweep40[1].data
# print('done')
# DECaLS_data41 = hdu_sweep41[1].data
# print('done')
# DECaLS_data42 = hdu_sweep42[1].data
# print('done')
# DECaLS_data43 = hdu_sweep43[1].data
# print('done')



done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [41]:
DECaLS_data = np.array([DECaLS_data1, 
DECaLS_data2,
DECaLS_data3, 
DECaLS_data4,
DECaLS_data5,
DECaLS_data6, 
DECaLS_data7,
DECaLS_data8,
DECaLS_data9,
DECaLS_data10,
DECaLS_data11,
DECaLS_data12,
DECaLS_data13,
DECaLS_data14, 
DECaLS_data15,
DECaLS_data16,
DECaLS_data17, 
DECaLS_data18]) #,
# DECaLS_data19,
# DECaLS_data20,
# DECaLS_data21,
# DECaLS_data22,
# DECaLS_data23,
# DECaLS_data24,
# DECaLS_data25,
# DECaLS_data26,
# DECaLS_data27,
# DECaLS_data28])


In [42]:
for i in range(len(DECaLS_data)):
    galdepth_g_BKG, galdepth_r_BKG, galdepth_z_BKG = read_data_new(SpecObj_data, SDSS_data, DECaLS_data[i])
        
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/raBKG/ra_BKG-220-240-dr9-{}.npz'.format(i), ra_BKG=ra_BKG)
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/decBKG/dec_BKG-220-240-dr9-{}.npz'.format(i), dec_BKG=dec_BKG)    
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/maskbitsBKG/maskbits_BKG-220-240-dr9-{}.npz'.format(i), maskbits_BKG=maskbits_BKG)
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/gfluxBKG/gflux_BKG-220-240-dr9-{}.npz'.format(i), gflux_BKG=gflux_BKG)
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/rfluxBKG/rflux_BKG-220-240-dr9-{}.npz'.format(i), rflux_BKG=rflux_BKG)
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/zfluxBKG/zflux_BKG-220-240-dr9-{}.npz'.format(i), zflux_BKG=zflux_BKG) 
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/w1fluxBKG/w1flux_BKG-220-240-dr9-{}.npz'.format(i), w1flux_BKG=w1flux_BKG)
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/w2fluxBKG/w2flux_BKG-220-240-dr9-{}.npz'.format(i), w2flux_BKG=w2flux_BKG)
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/w3fluxBKG/w3flux_BKG-220-240-dr9-{}.npz'.format(i), w3flux_BKG=w3flux_BKG)
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/w4fluxBKG/w4flux_BKG-220-240-dr9-{}.npz'.format(i), w4flux_BKG=w4flux_BKG)
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/mw_trans_g_BKG/mw_transmission_g_BKG-220-240-dr9-{}.npz'.format(i), mw_transmission_g_BKG=mw_transmission_g_BKG)
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/mw_trans_r_BKG/mw_transmission_r_BKG-220-240-dr9-{}.npz'.format(i), mw_transmission_r_BKG=mw_transmission_r_BKG)
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/mw_trans_z_BKG/mw_transmission_z_BKG-220-240-dr9-{}.npz'.format(i), mw_transmission_z_BKG=mw_transmission_z_BKG)
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/mw_trans_w1_BKG/mw_transmission_w1_BKG-220-240-dr9-{}.npz'.format(i), mw_transmission_w1_BKG=mw_transmission_w1_BKG)
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/mw_trans_w2_BKG/mw_transmission_w2_BKG-220-240-dr9-{}.npz'.format(i), mw_transmission_w2_BKG=mw_transmission_w2_BKG)
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/mw_trans_w3_BKG/mw_transmission_w3_BKG-220-240-dr9-{}.npz'.format(i), mw_transmission_w3_BKG=mw_transmission_w3_BKG)
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/mw_trans_w4_BKG/mw_transmission_w4_BKG-220-240-dr9-{}.npz'.format(i), mw_transmission_w4_BKG=mw_transmission_w4_BKG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-200/galdepth_g_BKG/galdepth_g_BKG-130-150-dr9-{}.npz'.format(i), galdepth_g_BKG=galdepth_g_BKG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-200/galdepth_r_BKG/galdepth_r_BKG-130-150-dr9-{}.npz'.format(i), galdepth_r_BKG=galdepth_r_BKG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-200/galdepth_z_BKG/galdepth_z_BKG-130-150-dr9-{}.npz'.format(i), galdepth_z_BKG=galdepth_z_BKG)
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/240-280/psfdepth_g_BKG/psfdepth_g_BKG-260-280-dr9-{}.npz'.format(i), psfdepth_g_BKG=psfdepth_g_BKG)
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/240-280/psfdepth_r_BKG/psfdepth_r_BKG-260-280-dr9-{}.npz'.format(i), psfdepth_r_BKG=psfdepth_r_BKG)
#     np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/240-280/psfdepth_z_BKG/psfdepth_z_BKG-260-280-dr9-{}.npz'.format(i), psfdepth_z_BKG=psfdepth_z_BKG)
    
    print('end saving files ', i)  
    
    

5789200
done reading in SDSS
5789200
done reading in DECaLS-SDSS matched file
4539426
done reading in DECaLS bricks
5789200
done creating unique IDs for matched file
length of id_ALL:  4539426
done creating unique IDs for brick file
27
4539426
27
end saving files  0
5789200
done reading in SDSS
5789200
done reading in DECaLS-SDSS matched file
3678246
done reading in DECaLS bricks
5789200
done creating unique IDs for matched file
length of id_ALL:  3678246
done creating unique IDs for brick file
0
3678246
0
end saving files  1
5789200
done reading in SDSS
5789200
done reading in DECaLS-SDSS matched file
998607
done reading in DECaLS bricks
5789200
done creating unique IDs for matched file
length of id_ALL:  998607
done creating unique IDs for brick file
0
998607
0
end saving files  2
5789200
done reading in SDSS
5789200
done reading in DECaLS-SDSS matched file
3821601
done reading in DECaLS bricks
5789200
done creating unique IDs for matched file
length of id_ALL:  3821601
done creating

In [60]:
# data0 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-0.npz')
# data1 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-1.npz')
# data2 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-2.npz')
# data3 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-3.npz')
# data4 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-4.npz')
# data5 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-5.npz')
# data6 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-6.npz')
# data7 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-7.npz')
# data8 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-8.npz')
# data9 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-9.npz')
# # data10 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-10.npz')
# # data11 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-11.npz')
# # data12 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-12.npz')
# # data13 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-13.npz')
# # data14 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-14.npz')
# # data15 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-15.npz')
# # data16 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-16.npz')
# # data17 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-17.npz')
# # data18 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-18.npz')
# # data19 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/psfdepth_z_BKG/psfdepth_z_BKG-350-360-dr9-19.npz')
# # data20 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/decBKG/dec_BKG-330-350-dr9-20.npz')
# # data21 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/decBKG/dec_BKG-330-350-dr9-21.npz')
# # data22 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/decBKG/dec_BKG-330-350-dr9-22.npz')
# # data23 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/decBKG/dec_BKG-330-350-dr9-23.npz')
# # data24 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-200/raBKG/ra_BKG-20-50-dr9-24.npz')
# # data25 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-200/raBKG/ra_BKG-20-50-dr9-25.npz')
# # data26 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-200/raBKG/ra_BKG-20-50-dr9-26.npz')
# # data27 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-200/raBKG/ra_BKG-20-50-dr9-27.npz')

# dataTOT = np.concatenate([data0['psfdepth_z_BKG'], data1['psfdepth_z_BKG'], data2['psfdepth_z_BKG'], data3['psfdepth_z_BKG'], data4['psfdepth_z_BKG'],
#                          data5['psfdepth_z_BKG'], data6['psfdepth_z_BKG'], data7['psfdepth_z_BKG'], data8['psfdepth_z_BKG'], data9['psfdepth_z_BKG']]) #,
# #                          data10['psfdepth_z_BKG'], data11['psfdepth_z_BKG'], data12['psfdepth_z_BKG'], data13['psfdepth_z_BKG'],
# #                          data14['psfdepth_z_BKG'], data15['psfdepth_z_BKG'], data16['psfdepth_z_BKG'], data17['psfdepth_z_BKG'], 
# #                          data18['psfdepth_z_BKG'], data19['psfdepth_z_BKG']]) #, data20['psfdepth_z_BKG'], data21['psfdepth_z_BKG'], 
# #                          data22['psfdepth_z_BKG'], data23['psfdepth_z_BKG'], data24['ra_BKG'], data25['ra_BKG'], data26['ra_BKG'], data27['ra_BKG']])

# print(len(dataTOT))

# # Save the new npz file
# np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/CATS/psfdepth_z_BKG-350-360-dr9.npz', dataTOT)


data0.close()
data1.close()
data2.close()
data3.close()
data4.close()
data5.close()
data6.close()
data7.close()
data8.close()
data9.close()
# data10.close()
# data11.close()
# data12.close()
# data13.close()
# data14.close()
# data15.close()
# data16.close()
# data17.close()
# data18.close()
# data19.close()
# data20.close()
# data21.close()
# data22.close()
# data23.close()
# data24.close()
# data25.close()
# data26.close()
# data27.close()


21590158


In [43]:
# combine the different arrays into a master catalog for each parameter

import numpy as np

data0 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/0-80/CATS/galdepth_z_BKG-0-20-dr9.npz')
data1 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/0-80/CATS/galdepth_z_BKG-20-50-dr9.npz')
data2 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/0-80/CATS/galdepth_z_BKG-50-80-dr9.npz')

data3 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-200/CATS/galdepth_z_BKG-100-130-dr9.npz')
data4 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-200/CATS/galdepth_z_BKG-130-150-dr9.npz')
data5 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-200/CATS/galdepth_z_BKG-150-170-dr9.npz')
data6 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-200/CATS/galdepth_z_BKG-170-190-dr9.npz')
data7 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-200/CATS/galdepth_z_BKG-190-200-dr9.npz')

data8 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/CATS/galdepth_z_BKG-200-220-dr9.npz')
data9 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/200-240/CATS/galdepth_z_BKG-220-240-dr9.npz')

data10 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/240-280/CATS/galdepth_z_BKG-240-260-dr9.npz')
data11 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/240-280/CATS/galdepth_z_BKG-260-280-dr9.npz')

data12 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/CATS/galdepth_z_BKG-300-330-dr9.npz')
data13 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/CATS/galdepth_z_BKG-330-350-dr9.npz')
data14 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/CATS/galdepth_z_BKG-350-360-dr9.npz')



totalDAT = np.concatenate([data0['arr_0'], data1['arr_0'], data2['arr_0'], data3['arr_0'], data4['arr_0'], 
            data5['arr_0'], data6['arr_0'], data7['arr_0'], data8['arr_0'], data9['arr_0'], data10['arr_0'],
            data11['arr_0'], data12['arr_0'], data13['arr_0'], data14['arr_0']])


np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/galdepth_z_BKG-TOTAL-dr9.npz', totalDAT)

print(len(totalDAT))



500760241


In [60]:
data = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/galdepth_z_BKG-TOTAL-dr9.npz')
xtra = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/galdepth_z_BKG-xtra-dr9.npz')

totalDATA = np.concatenate([data['arr_0'], xtra['arr_0']])

np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/galdepth_z_BKG-TOTAL-dr9.npz', totalDATA)

print(len(totalDATA))


505492799


In [34]:
488908775

galdepth = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/CATS/galdepth_z_BKG-350-360-dr9.npz')
ra = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/300-360/CATS/ra_BKG-350-360-dr9.npz')

print(len(galdepth['arr_0']))
print(len(ra['arr_0']))

# print(len(ra['arr_0']) - len(galdepth['arr_0']))

# print()
# print()

# galdepthTOT = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/galdepth_z_BKG-TOTAL-dr9.npz')
# raTOT = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/ra_BKG-TOTAL-dr9.npz')

# print(len(galdepthTOT['arr_0']))
# print(len(raTOT['arr_0']))
# print(len(raTOT['arr_0']) - len(galdepthTOT['arr_0']))


20320226
21590158


In [46]:
# bin0 == 0 < ra < 40   -15 < dec < 10
# bin1 == 90 < ra < 210   -10 < dec < 30
# bin2 == 240 < ra < 250   0 < dec < 20
# bin3a == 320 < ra < 330   -15 < dec < 10
# bin3b == 330 < ra < 340   10 < dec < 30
# bin3c == 350 < ra < 360   -15 < dec < 20

raBKG_data = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/ra_BKG-TOTAL-dr9.npz')
raBKG = raBKG_data['arr_0']
print(len(raBKG))

decBKG_data = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/dec_BKG-TOTAL-dr9.npz')
decBKG = decBKG_data['arr_0']
print(len(decBKG))

galdepth_z_BKG_data = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/galdepth_z_BKG-TOTAL-dr9.npz')
galdepth_z_BKG = galdepth_z_BKG_data['arr_0']
print(len(galdepth_z_BKG))



500760241
500760241
500760241


In [48]:
raBKG_0to40 = raBKG[np.where((raBKG >= 0.) & (raBKG <= 40.) & (decBKG >= -15.) & (decBKG <= 10.))]
decBKG_0to40 = decBKG[np.where((raBKG >= 0.) & (raBKG <= 40.) & (decBKG >= -15.) & (decBKG <= 10.))]
galdepth_z_BKG_0to40 = galdepth_z_BKG[np.where((raBKG >= 0.) & (raBKG <= 40.) & (decBKG >= -15.) & (decBKG <= 10.))]
print(len(raBKG_0to40))
print(len(decBKG_0to40))
print(len(galdepth_z_BKG_0to40))
print()

raBKG_90to210 = raBKG[np.where((raBKG >= 90.) & (raBKG <= 210.) & (decBKG >= -10.) & (decBKG <= 30.))]
decBKG_90to210 = decBKG[np.where((raBKG >= 90.) & (raBKG <= 210.) & (decBKG >= -10.) & (decBKG <= 30.))]
galdepth_z_BKG_90to210 = galdepth_z_BKG[np.where((raBKG >= 90.) & (raBKG <= 210.) & (decBKG >= -10.) & (decBKG <= 30.))]
print(len(raBKG_90to210))
print(len(decBKG_90to210))
print(len(galdepth_z_BKG_90to210))
print()

raBKG_240to250 = raBKG[np.where((raBKG >= 240.) & (raBKG <= 250.) & (decBKG >= 0.) & (decBKG <= 20.))]
decBKG_240to250 = decBKG[np.where((raBKG >= 240.) & (raBKG <= 250.) & (decBKG >= 0.) & (decBKG <= 20.))]
galdepth_z_BKG_240to250 = galdepth_z_BKG[np.where((raBKG >= 240.) & (raBKG <= 250.) & (decBKG >= 0.) & (decBKG <= 20.))]
print(len(raBKG_240to250))
print(len(decBKG_240to250))
print(len(galdepth_z_BKG_240to250))
print()

raBKG_320to330 = raBKG[np.where((raBKG >= 320.) & (raBKG <= 330.) & (decBKG >= -15.) & (decBKG <= 10.))]
decBKG_320to330 = decBKG[np.where((raBKG >= 320.) & (raBKG <= 330.) & (decBKG >= -15.) & (decBKG <= 10.))]
galdepth_z_BKG_320to330 = galdepth_z_BKG[np.where((raBKG >= 320.) & (raBKG <= 330.) & (decBKG >= -15.) & (decBKG <= 10.))]
print(len(raBKG_320to330))
print(len(decBKG_320to330))
print(len(galdepth_z_BKG_320to330))
print()

raBKG_330to340 = raBKG[np.where((raBKG >= 330.) & (raBKG <= 340.) & (decBKG >= 10.) & (decBKG <= 30.))]
decBKG_330to340 = decBKG[np.where((raBKG >= 330.) & (raBKG <= 340.) & (decBKG >= 10.) & (decBKG <= 30.))]
galdepth_z_BKG_330to340 = galdepth_z_BKG[np.where((raBKG >= 330.) & (raBKG <= 340.) & (decBKG >= 10.) & (decBKG <= 30.))]
print(len(raBKG_330to340))
print(len(decBKG_330to340))
print(len(galdepth_z_BKG_330to340))
print()

raBKG_350to360 = raBKG[np.where((raBKG >= 350.) & (raBKG <= 360.) & (decBKG >= -15.) & (decBKG <= 20.))]
decBKG_350to360 = decBKG[np.where((raBKG >= 350.) & (raBKG <= 360.) & (decBKG >= -15.) & (decBKG <= 20.))]
galdepth_z_BKG_350to360 = galdepth_z_BKG[np.where((raBKG >= 350.) & (raBKG <= 360.) & (decBKG >= -15.) & (decBKG <= 20.))]
print(len(raBKG_350to360))
print(len(decBKG_350to360))
print(len(galdepth_z_BKG_350to360))
print()





65253316
65253316
65253316

165175758
165175758
165175758

8321443
8321443
8321443

12303351
12303351
12303351

7529769
7529769
7529769

16738686
16738686
16738686



In [49]:
np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/raBKG_0to40-dr9.npz', raBKG_0to40)
np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/decBKG_0to40-dr9.npz', decBKG_0to40)
np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/galdepth_z_BKG_0to40-dr9.npz', galdepth_z_BKG_0to40)

np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/raBKG_90to210-dr9.npz', raBKG_90to210)
np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/decBKG_90to210-dr9.npz', decBKG_90to210)
np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/galdepth_z_BKG_90to210-dr9.npz', galdepth_z_BKG_90to210)

np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/raBKG_240to250-dr9.npz', raBKG_240to250)
np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/decBKG_240to250-dr9.npz', decBKG_240to250)
np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/galdepth_z_BKG_240to250-dr9.npz', galdepth_z_BKG_240to250)

np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/raBKG_320to330-dr9.npz', raBKG_320to330)
np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/decBKG_320to330-dr9.npz', decBKG_320to330)
np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/galdepth_z_BKG_320to330-dr9.npz', galdepth_z_BKG_320to330)

np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/raBKG_330to340-dr9.npz', raBKG_330to340)
np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/decBKG_330to340-dr9.npz', decBKG_330to340)
np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/galdepth_z_BKG_330to340-dr9.npz', galdepth_z_BKG_330to340)

np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/raBKG_350to360-dr9.npz', raBKG_350to360)
np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/decBKG_350to360-dr9.npz', decBKG_350to360)
np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/BKG/galdepth_z_BKG_350to360-dr9.npz', galdepth_z_BKG_350to360)

